<a href="https://colab.research.google.com/github/PrathamKumar125/Invoice_information_Extraction/blob/master/Invoice_information_Extraction_LLAMA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Install all the packages

In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes sentence_transformers llama-index pypdf python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
!pip install -q langchain-community llama_index.embeddings.langchain llama_index.llms.huggingface

In [12]:
!pip install -q pytesseract PyMuPDF Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 67.1 MB/s eta 0:00:00


In [7]:
!sudo apt-get update
!sudo apt-get install tesseract-ocr

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2,954 kB]
Ign:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [51.8 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,422 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,441 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 

# 2. Packages imports

In [13]:
import sys
import torch
from pprint import pprint
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import ServiceContext
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.core import ChatPromptTemplate

# 3. Login to HuggingFace cli

In [9]:
!git config --global credential.helper store
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# 4. Load the Dataset

In [10]:
!mkdir data

In [14]:
import fitz
from PIL import Image
import numpy as np
import os
import pytesseract

def extract_text_from_pdf(pdf_path):
    text = ""
    document = fitz.open(pdf_path)
    for page in document:
        text += page.get_text()
    return text

def extract_text_from_image(image_path):
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    return text

def load_data_from_directory(data_location):
    documents = []
    for filename in os.listdir(data_location):
        file_path = os.path.join(data_location, filename)
        if filename.lower().endswith('.pdf'):
            text = extract_text_from_pdf(file_path)
            documents.append({'filename': filename, 'text': text})
        elif filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            text = extract_text_from_image(file_path)
            documents.append({'filename': filename, 'text': text})
    return documents

In [15]:
data_location = '/content/data'
documents = load_data_from_directory(data_location)

In [16]:
documents[0]

{'filename': 'Sample Invoice.pdf',
 'text': "TATA MOTORS LIMITED\nTATA MOTORS LIMITED Nigadi Bhosari Road, PIMPRI\nPune, MAHARASHTRA- 411018 \nGSTIN: 27AAACT2727Q1ZW    \nSeeds Lic. No: LASD67898369\nFertilizers Lic. No: LASD34564756\nInsecticide Lic. No: LAID 26453734\nT A X  I N V O I C E\nORIGINAL FOR RECIPIENT\nCustomer Details:\nTEST\nBilling Address:\nTest\nHyderabad, TELANGANA, 500089\nPh: 9108239284\ntest@gmail.com\nShipping Address:\nTest\nHyderabad, TELANGANA, 500089\nInvoice #: \ninv41\nDate: \n18 Jul 2024\nPlace of Supply: \n36-TELANGANA \nEnquire id: \n06-06-2024\nIMEI \nNO\nITEM / Calibration Charges For\nHSN/ SAC\nRate\nQuantity\nAmount\nIGST\nTotal Amount\n1\n WASTE AND SCRAP OF STAINLESS \nSTEEL \n72042190\n95.00\n6,790 KGS\n6,45,050.00\n1,16,109.00 \n(18%)\n7,61,159.00\n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n 

In [17]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a data extractor. Extract the exact data from given document. If no information found please reply 'no information available."
            )
        ),
        HumanMessagePromptTemplate.from_template("{query_str}"),
    ]
)

In [18]:
system_prompt = "You are a data extractor. Extract the exact data from given document. If no information found please reply 'no information available."

# 5. Load the Embeddings

In [19]:
# hyperparameters
context_window = 4096
temperature = 0.0
model_name = 'meta-llama/Llama-2-7b-chat-hf'

In [20]:
llm = HuggingFaceLLM(
    context_window=context_window,
    max_new_tokens=256,
    generate_kwargs={"temperature": temperature, "do_sample": False},
    system_prompt=system_prompt,
    tokenizer_name=model_name,
    model_name=model_name,
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True, "use_auth_token":True},

)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [21]:
embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [22]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

<ipython-input-22-6394536e40c5>:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


#6. Index the datastore

In [28]:
from llama_index.core import Document

# Extract text values from the dictionaries
documents = [Document(text=d.get('text', '')) for d in documents]

index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [30]:
user_queries = ['Customter details?' , 'Products?', 'Total Amount?' ]

In [31]:
answer = dict()
for i, user_query in enumerate(user_queries):
    query_engine = index.as_query_engine()
    response = query_engine.query(user_query)
    answer.update({user_query: response.response})

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [32]:
print(answer)

{'Customter details?': '\nThe customer details are:\n\n* Name: TEST\n* Address: Test, Hyderabad, TELANGANA, 500089\n* Phone: 9108239284\n* Email: test@gmail.com\n\nNote: The customer details are mentioned in the invoice under the "Bill To" section.', 'Products?': '\n1. WASTE AND SCRAP OF STAINLESS STEEL\nHSN: 72042190\nQuantity: 6,790 KGS\nRate: 95.00\nTotal Amount: 6,45,050.00\n\n2. IMEI NO\nITEM / Calibration Charges For\nHSN/ SAC\nRate\nQuantity\nAmount\nIGST\nTotal Amount\n1\n WASTE AND SCRAP OF STAINLESS STEEL \n72042190\n95.00\n6,790 KGS\n6,45,050.00\n1,16,109.00 \n(18%)\n7,61,159.00\n\nPlease let me know if you need any further assistance.', 'Total Amount?': '₹7,68,771.00'}


In [33]:
import json

file_name = "output.txt"

with open(file_name, 'w') as file:
    json.dump(answer, file)

print(f"Content saved to {file_name}")

Content saved to output.txt
